In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import time, random
import multiprocessing
from multiprocessing import Pool
from contextlib import closing
from functools import partial
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve,auc,precision_recall_curve
from scipy.stats import mode
from sklearn.externals import joblib
import matplotlib.pyplot as plt

In [24]:
def rf_classifier(feat):
    t=time.time()
    '''
    max_features,n_estimators = feat
    X_train = pd.read_csv('X_train.csv',sep=',',header=0).values
    y_train = pd.read_csv('y_train.csv',sep=',',header=0).values
    X_pos = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==1)])
    y_pos = y_train[y_train==1]
    X_neg = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==0)])
    y_neg = y_train[y_train==0]
    over_samp_ct = 100000
    _,d = X_pos.shape
    n_pos_train = X_pos.shape[0]
    n_neg_train = X_neg.shape[0]
    Over_under_X_train = np.zeros([over_samp_ct*2,d])
    Over_under_y_train = np.zeros([over_samp_ct*2,])
    for i in range(over_samp_ct):
        idx_pos = random.randint(0,n_pos_train-1)
        idx_neg = random.randint(0,n_neg_train-1)
        Over_under_X_train[2*i,:] = X_pos[idx_pos,:]
        Over_under_y_train[2*i] = y_pos[idx_pos]
        Over_under_X_train[2*i+1,:] = X_neg[idx_neg,:]
        Over_under_y_train[2*i+1] = y_neg[idx_neg]
    #rf = RandomForestClassifier(n_estimators=n_estimators,max_features=max_features).fit(Over_under_X_train,Over_under_y_train)
    '''
    rf = joblib.load('Random_forest_classifier_best.sav')
    X_pos = []
    y_pos = []
    X_neg = []
    y_neg = []
    Over_under_X_train = []
    Over_under_y_train = []
    X_valid = pd.read_csv('X_valid.csv',sep=',',header=0).values
    y_valid = pd.read_csv('y_valid.csv',sep=',',header=None).values
    #train_pred = rf.predict(X_train)
    valid_pred = rf.predict_proba(X_valid)[:,1]
    X_train = []
    #fpr_tr, tpr_tr, thresh_tr = roc_curve(y_train, train_pred)
    fpr_vl, tpr_vl, thresh_vl = roc_curve(y_valid, valid_pred)
    pr,rl,tr = precision_recall_curve(y_valid,valid_pred)
    #auc_tr = auc(fpr_tr,tpr_tr)
    auc_vl = auc(fpr_vl,tpr_vl)
    #pr_ar = auc(pr,rl)
    pr_vl = pr[np.where(rl>0.9)[0][-1]]
    print('rf',precision_score(y_valid,rf.predict(X_valid)),recall_score(y_valid,rf.predict(X_valid)))
    print("threshold: "+str(tr[np.where(rl>0.9)[0][-1]]))
    #print("rf:",str(pr[np.where(tr>0.5)[0][-1]]),str(rl[np.where(tr>0.5)[0][-1]]))
    '''
    plt.figure()
    plt.plot(fpr_vl,tpr_vl)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("AUC for best Random Forest Model: "+str(np.round(auc_vl,4)))
    plt.savefig('AUC_for_best_Random_Forest_Model.png')
    plt.figure()
    plt.plot(rl,pr)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall curve for best Random Forest Model")
    plt.savefig('PR_curve_for_best_Random_Forest_Model.png')
    print(auc_vl,pr_vl)
    #joblib.dump(rf,'Random_forest_classifier_best.sav')
    '''

In [25]:
def gbm_classifier(feat):
    t=time.time()
    '''
    max_features,max_depth,learning_rate,n_estimators = feat
    X_train = pd.read_csv('X_train.csv',sep=',',header=0).values
    y_train = pd.read_csv('y_train.csv',sep=',',header=0).values
    X_pos = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==1)])
    y_pos = y_train[y_train==1]
    X_neg = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==0)])
    y_neg = y_train[y_train==0]
    over_samp_ct = 100000
    _,d = X_pos.shape
    n_pos_train = X_pos.shape[0]
    n_neg_train = X_neg.shape[0]
    Over_under_X_train = np.zeros([over_samp_ct*2,d])
    Over_under_y_train = np.zeros([over_samp_ct*2,])
    for i in range(over_samp_ct):
        idx_pos = random.randint(0,n_pos_train-1)
        idx_neg = random.randint(0,n_neg_train-1)
        Over_under_X_train[2*i,:] = X_pos[idx_pos,:]
        Over_under_y_train[2*i] = y_pos[idx_pos]
        Over_under_X_train[2*i+1,:] = X_neg[idx_neg,:]
        Over_under_y_train[2*i+1] = y_neg[idx_neg]
    rf = GradientBoostingClassifier(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth,learning_rate=learning_rate).fit(Over_under_X_train,Over_under_y_train)
    '''
    rf = joblib.load('GBM_classifier_best.sav')
    X_pos = []
    y_pos = []
    X_neg = []
    y_neg = []
    Over_under_X_train = []
    Over_under_y_train = []
    X_valid = pd.read_csv('X_valid.csv',sep=',',header=0).values
    y_valid = pd.read_csv('y_valid.csv',sep=',',header=None).values
    #train_pred = rf.predict(X_train)
    valid_pred = rf.predict_proba(X_valid)[:,1]
    X_train = []
    #X_valid = []
    #fpr_tr, tpr_tr, thresh_tr = roc_curve(y_train, train_pred)
    fpr_vl, tpr_vl, thresh_vl = roc_curve(y_valid, valid_pred)

    #auc_tr = auc(fpr_tr,tpr_tr)
    auc_vl = auc(fpr_vl,tpr_vl)
    
    pr,rl,tr = precision_recall_curve(y_valid,valid_pred)
    #pr_ar = auc(pr,rl)
    pr_vl = pr[np.where(rl>0.9)[0][-1]]
    print('gbm',precision_score(y_valid,rf.predict(X_valid)),recall_score(y_valid,rf.predict(X_valid)))
    print("threshold: "+str(tr[np.where(rl>0.9)[0][-1]]))
    #print("gbm:",str(pr[np.where(tr>0.5)[0][-1]]),str(rl[np.where(tr>0.5)[0][-1]]))
    '''
    plt.figure()
    plt.plot(fpr_vl,tpr_vl)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("AUC for best GBM Model: "+str(np.round(auc_vl,4)))
    plt.savefig('AUC_for_best_GBM_Model.png')
    plt.figure()
    plt.plot(rl,pr)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall curve for best GBM Model")
    plt.savefig('PR_curve_for_best_GBM_Model.png')
    print(auc_vl,pr_vl)
    #joblib.dump(rf,'GBM_classifier_best.sav')
    '''

In [26]:
def lr_classifier(feat):
    t=time.time()
    '''
    C = feat[0]
    X_train = pd.read_csv('X_train.csv',sep=',',header=0).values
    y_train = pd.read_csv('y_train.csv',sep=',',header=0).values
    X_pos = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==1)])
    y_pos = y_train[y_train==1]
    X_neg = np.array([X_train[i,:] for i in range(X_train.shape[0]) if (y_train[i]==0)])
    y_neg = y_train[y_train==0]
    over_samp_ct = 100000
    _,d = X_pos.shape
    n_pos_train = X_pos.shape[0]
    n_neg_train = X_neg.shape[0]
    Over_under_X_train = np.zeros([over_samp_ct*2,d])
    Over_under_y_train = np.zeros([over_samp_ct*2,])
    for i in range(over_samp_ct):
        idx_pos = random.randint(0,n_pos_train-1)
        idx_neg = random.randint(0,n_neg_train-1)
        Over_under_X_train[2*i,:] = X_pos[idx_pos,:]
        Over_under_y_train[2*i] = y_pos[idx_pos]
        Over_under_X_train[2*i+1,:] = X_neg[idx_neg,:]
        Over_under_y_train[2*i+1] = y_neg[idx_neg]
    rf = LogisticRegression(C=C,penalty='l1').fit(Over_under_X_train,Over_under_y_train)
    '''
    rf = joblib.load('lr_classifier_best.sav')
    X_pos = []
    y_pos = []
    X_neg = []
    y_neg = []
    Over_under_X_train = []
    Over_under_y_train = []
    X_valid = pd.read_csv('X_valid.csv',sep=',',header=0).values
    y_valid = pd.read_csv('y_valid.csv',sep=',',header=None).values
    #train_pred = rf.predict_proba(X_train)
    valid_pred = rf.predict_proba(X_valid)[:,1]
    X_train = []
    X_valid = []
    #fpr_tr, tpr_tr, thresh_tr = roc_curve(y_train, train_pred)
    fpr_vl, tpr_vl, thresh_vl = roc_curve(y_valid, valid_pred)
    
    pr,rl,tr = precision_recall_curve(y_valid,valid_pred)
    
    #pr_ar = auc(pr,rl)
    pr_vl = pr[np.where(rl>0.9)[0][-1]]
    #auc_tr = auc(fpr_tr,tpr_tr)
    auc_vl = auc(fpr_vl,tpr_vl)
    plt.figure()
    plt.plot(fpr_vl,tpr_vl)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("AUC for best Logestic Regression Model: "+str(np.round(auc_vl,4)))
    plt.savefig('AUC_for_best_Logestic_Regression_Model.png')
    plt.figure()
    plt.plot(rl,pr)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall curve for best Logestic Regression Model")
    plt.savefig('PR_curve_for_best_Logestic_Regression_Model.png')
    print(auc_vl,pr_vl)
    #joblib.dump(rf,'lr_classifier_best.sav')

In [27]:
def which_class(ft):
    cls,*ft = ft
    if cls=='lr':
        lr_classifier(ft)
    elif cls=='gbm':
        gbm_classifier(ft)
    elif cls=='rf':
        rf_classifier(ft)

In [28]:
feat = [['rf',28,900],['gbm',33,3,0.1,500],['lr',10]]
with closing(Pool(3)) as pool:
    pool.map(which_class,feat)

0.9837630268262498 0.26888139112379966
gbm 0.17704556203689106 0.9756733275412685
threshold: 0.879108024905775
rf 0.24988078206962328 0.9105125977410947
threshold: 0.5211111111111111


In [6]:
cls,*args = ['lr',33,3,0.1,500]

In [7]:
args

[33, 3, 0.1, 500]

In [20]:
np.round(0.98765,3)

0.988